## Table of Contents (edit this last)
* [Module 5: Model Based Process Control](#Module-5:-Model-Based-Process-Control)
    * [Concepts](#Concepts:)
    * [User Stories](#User-Stories:)
        * [Model Predictive Control](2.%20Learning%20Modules/2.6.%20Model%20Predictive%20Control/Model%20Predictive%20Control.ipynb)
    * [Assessment](#Assessment:)
        * [Take a quiz](http://example.com/quiz)
#### [🏠 Home](../../welcomePage.ipynb)

# Motivation
Like layers of an onion, production systems have multiple layers with different timescales relevant to each layer. In the production of a complex system of assebmlies of parts, such as a ground vehicle, the innermost layer is the workpiece layer at which each workpiece is shaped. Shaping processes include casting, forging, extrusion, forming, stamping, deposition, and machining. In fact, a combination of processes in sequence is often used to shape the workpiece to its final shape. At the workpiece layer of production, events at the millisecond time scale affect the quality and cost of production. Hence, sensor data should be collected at the millisecond time scale as well for process improvement.
<center>
    <img src="../../img/Process Layers of Production Technology Fig 11.1.svg" alt="The layers of production are shown as concentric rings with the workpiece at the center and the assembly of workpieces as the next ring out from the center." width="600">
</center>
The next layer outside of the workpiece layer is the assembly layer in which workpieces are joined into an assembly. The timescale of events and their data relevance is typically larger at this layer, for example, a timescale of minutes, although precise millisecond data my be releveant during brief events of joining and fastening. This module will not cover the larger layers of the complete system/product and its usage in the field with their associated longer event and data timescales.

With these layers of production in mind, we turn to the task of using data collected at the millisecond-to-minute time scale to improve the processes of workpiece shaping and assembly joining, realizing that digitally actuated machine tools can run more optimally simply by making digital changes (modified software, modified parameter files) in an improvement cycle that far exceeds the rate of learning of a human operator, for example, a welder executing a repetitive manual operation. For automated production processes, we can construct "gray box" models for direct digital control of a process using combinations of technical information and raw data from the production process. Furthermore, this approach can provide useful information to humans for the critical tasks of process planning, production management, and production development.

Our schematic for this modle based process control is shown below. We will see how a single machine on a shop floor can participate in the optimization of model based process control, and then how the data and experiences of one machine can enable improvements across all the machines performing this process within a network or "World Wide Lab" of connected shop floors.

<center>
    <img src="../../img/Model Based Control System for Manufacturing Process.svg" alt="This schematic of a model based control system for a manufacturing process shows a typical closed loop control system diagram with a sensed output signal fed back to control. Important additions are the collection of data from the Decision & Control block as well as the Machine block. Furthermore, the control section of the diagram is broken into a sequence of three blocks: Model Identification, Model Optimization, and Decision & Control" width="800">
</center>

# <font color = 'red'>1. *Introduction to Model Based Process Control*

Objective:  Understand how process data can be collected and used to improve performance.
(Introduction to process data collection methods, such as sensors.)

## *1.1 Sensing and Data Aggregation*
Machine tools typically have integrated sensors in their design and often have the ability to acquire and transmit data from these onboard sensors. For example, a CNC (computer numerically controlled) machining center typically has sensing through linear or rotary position encoders on its motion stages, a spindle speed sensor, and sensors on the amount of electric current applied to each motor (a signal proportional to force in linear motion or torque in rotary motion). The process data on the motor drive's internal microcontroller registers can also be collected such as the tool number of the active tool, the present line number of the NC program, the zero point offset, and the sensed position and commanded position and their difference (the position error). Similarly, velocity and acceleration errors can be collected as well. More advanced machines may also include thermal, acoustic, and vibrational sensing and machine vision, or aftermarket sensor packages can be added to a machine to collect this as well. All of this data may be collected as fast as the position control cycle (1000 Hz gives a 1 millisecond sampling interval). One challenge, however, is the lack of an industry-wide standard for the sensor data interface and data format. Middleware software providers offer solutions to collect the data using the available interface of the machine tool with an additional computer.

Data standards are as follows. Low-frequency dimensional data may be captured using the standardized OPC UA (Open Platform Communications Unified Architecture) interface for a machine.
Once read, the machine data can be sent through a network using a publish/subscribe protocol such as MQTT (Message Queuing Telemetry Transport). The midddleware annotates machine information with metadata for automatic structuring and storage in a database.

In addition to these integrated sensors on the machine tool, sensing is available in stanadard metrology and quality inspection steps which involve transporting the workpiece from the machine tool to a metrology lab. While metrology data collected according to ISO standards on a coordinate measuring machine is both accurate and precise, the time and inconvenience of moving the workpiece to a climate controlled lab and waiting for the workpiece to reach the standard specified temperature are good reasons to seek faster measurement options. CNC machines have probing systems which can give on-machine workpiece geometry measurements. These on-machine measurements will have higher error than CMM measurements due to both thermal expansion and geometric errors of the CNC machine tool.se

## *1.2 Definition of Process Control*

We use process control to manage and maintain the quality, efficiency, and consistency of manufacturing processes. 
The goal is to ensure that products are produced within specification, while minimizing variability and defects, and maximizing throughput or otherwise reducing cost of production.

Process control has two main approaches:
1. **Statistical Process Control**: In this approach, a typically well-controlled process can be monitored for deviation and adjustments made to bring the process back within control limits. As its name implies, statistical models are used to mathematically describe the process from measured process data. It is these statistical models that enable the decision-making of selecting the best course of action to bring the process to the desired state.
2. **Dynamic System Control**: Alternatively, this approach uses the mathematics of differential equations and dynamic systems to describe system behavior in order to control system behavior.
Dynamic systems are those for which the state or status of the system changes in time according to some mathematical rule.
Typically, the output can be predicted from the input, with some level of uncertainty, by applying the known mathmatical rules, called the system model.
System models employ differential equations in which the time derivative (the rate-of-change) of a state variable is a known function of that state variable and the other state variables and inputs of the system.

Some examples of dynamic control are:
- **PID Controllers**: Pronounced "P. I. D.", these controllers use an error signal and its integral and derivative to generate an input actuation to the process under control. Typically they are tuned once and set for implementation to drive the output to a new or changing setpoint during the process.
- **Model-based Optimal Controllers**: Typically linear dynamical system models are used to describe the behavior of a system at operating levels near the desired operation point. These models have some error as physical processes often have significant nonlinearity. The benefit of using linear models is that optimization theory has already been developed, giving controllers which can drive a linear system to the desired state under some optimization such as minimum time or minimum energy expenditure. An optimal controller can exceed the performance of a PID controller when a well-identified linear model is used to compose the optimal controller, essentially because the optimal controller has more levers for tuning its performance.
- **Model-based Robust Controllers**: When the process under control exhibits variations such as disturbances due to additional physical inputs, a controller can lose performance. For example, a process which varies with ambient temperature will suffer if it uses the same PID controller or model-based optimal controller despite significant variation in ambient temperature. Essentially the variation represents an aspect of the process that is not captured in the linear model that was used to tune the optimal controller. One approach to handle expected variations is to model the amount of variation in the process behavior. With the combination of a linear model and its uncertainty, an optimal controller can be constructed to optmize for its performance across a range of operating conditions. The end result is a tuned controller which work can work well enough over a range of operating conditions. We call this robust as the controller tuning is robust to a specified level of variations, noise, or disturbance.
- **Model Predictive Controllers**: The variations in a process due to uncontrolled inputs or changes in the states can be modeled. If such a model is constructed and has low error, it can be used in this advanced form of control. The model predictive controller will use a computer simulation of a short term horizon of the behavior of the system. With this simulation, the controller will solve an optimization problem to determine the best sequence of actuations to drive the output to the desired state within the time horizon. Once that optimization is solved, the next actuation value is taken from the solution to the optimization problem and used as the actuation of the physcial process. This process of prediction and optimizaiton for actuation is repeted at each timestep. This controller works well even for somewhat nonlinear processes. The cost of so much comptuation is the requirement for a capable microprocessor embedded close to the system, which can be well worth the expense.
 
The key to both the statistical and dynamic system approaches is a mathematical model, and both approaches benefit from statistical analysis of uncertainty. We will explore further the models used in process control in order to understand their limits and how big data is enabling better performance than ever in model based process control.

In Dynamic System Control, a control system can be mathematically formulated in order to optimize system performance.
The control system can be simulated with a system model to validate its performance in a computer simulation.
Later, the control system can be tested with a mixture of hardware and computer simulations in a procedure called "hardware-in-the-loop" (HIL) testing.
The physical hardware can include control electronics, actuators, and sensors. The computer models can simulate parts of the manufacturing system which are more expensive to prototype, have challenging safety concerns, or have a longer lead time.
The benefit of HIL testing is to further validate the control system implementation, now admitting effects that were not captured in the earlier simulations without hardware. These effects include the conversions between analog signals and digital signals, time delays, and physical interactions such as electromagnetic interference.
HIL testing is still less expensive than full scale testing which requires the investment in the process and feedstock materials and produces waste when the process outputs are not in the specified tolerance. 

## *1.2 Example of Manufacturing Process Control*

A manufacturing process example will be selected with an accurate nonlinear, noisy model of the process.
A process setpoint will be selected.
Designed controllers of a few types will be demonstrated: PID, Optimal, Robust, and MPC.
The error signal will be plotted.
The area under the error signal curve will be shaded and a bar chart plotted.
The controller with the least error will be identified.

The plots will be repeated for a different setpoint with the controller.
This should show the robust controller outperforming the optimal controller.
The model predictive controller should have good performance in each case. 

Remaining Outline:

-  See a variety of controllers drive error to zero-  	See noise/disturbance/model uncertainty-  •	See data used to identify/fit/train a mode-  
•	True process will have time dependent degradation (mass, damping, etc) plus temp dependent behavior (time since startup, 1st order response, 70 minute time constant), plus temp depdented behavior to environment temp (trapezoidal wave for HVAC system) plus random noise
•	Train using data from day 1 or machie  1
•	Train using a dayslys  ta
•	Train using last 1 hour data, evaluate controller performa

Use example of a robotic gripper
Frequency domain controller - designed with specifications.
Alternative - add a sensor ( a potentiometer with DAQ), use full state feedback control - get better performance.
explore the sensor and DAQ setup, concepts of resolution, sample rate, performance hits for these.nce


### Module 5: Model Based Process Control

Model-based process control is a mathematical and visual technique to addressing the challenges associated with designing complex control. This technique involves two key aspects. (a) The control analysis, representing the modeling and simulation aspects of the system including tuning and configuration, and automatic code generation. Each block of the model contains mathematical equations that emulate the behavior of the physical system. (b) The system and software, representing continuous validation and verification through testing, and deployment of the system. In this regard, when model-based control is used efficiently, it offers a single design platform to enhance overall system design. Through virtual prototyping, system engineers can easily see whether the whole system is working as intended, even before the hardware is manufactured and available for testing.


#### Concepts:
<center>
    <img src="../../img/03.jpg" alt="Alt text" width="400">
</center>


####  User Stories:
- [Model Predictive Control](2.%20Learning%20Modules/2.6.%20Model%20Predictive%20Control/Model%20Predictive%20Control.ipynb) - This module demonstrates calculating a single time step's move trajectory using the same system as the Dahlin controller.

#### Assessment:
- [Take a quiz](http://example.com/quiz)

### <center>[◀︎ Module 4](Module4.ipynb)     [🏠 Home](../../welcomePage.ipynb)     [Module 6 ▶︎](Module6.ipynb)</center>
